# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment : IoT and Edge Devices - CNN to classify Cifar 10 dataset using TFLite

### Learning Objectives:

At the end of the experiment, you will be able to:

*  train and evaluate a CNN model on CIFAR10 dataset using Tensorflow
*  convert a Tensorflow model to TensorflowLite for Microcontrollers
*  test the TFLite Model using TFLite Interpreter
*  understand the types of optimization in TFLite - quantization, pruning and clustering
*  prune and fine-tune the model to 50% sparsity
*  apply sparsity preserving clustering on the pruned model and observe the number of clusters and check that the sparsity is preserved
*  apply both QAT and PCQAT on the sparse clustered model
*  evaluate the model, which has been pruned, clustered and quantized and then check the accuracy from TensorFlow persists in the TFLite backend.


## Dataset

#### Description

In this experiment, we will use the CIFAR-10 dataset from keras API. It consists of 60,000 colour images(32x32) in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images.



Here are the classes in the dataset, as well as 10 random images from each:


<img src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Images/CIFAR10.png" alt="Drawing" height="350" width="440"/>


### CNN to classify Cifar-10 dataset (Images)



So far, we saw how to build a Dense Neural Network (DNN) that classified images of digits (MNIST) or even fashion images (Fashion-MNIST). Here we will instead, recognize the 10 classes of CIFAR ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' and 'truck'). There are some key differences between these two image datasets that we need to take into account.

First, while MNIST were 28x28 monochrome images (1 color channel), CIFAR is 32x32 color images (3 color channels).

Second, MNIST images are simple, containing just the object centered in the image, with no background. Conversely, CIFAR ones are not centered and can have the object with a background, such as airplanes that might have a cloudy sky behind them! Those differences are the main reason to use a CNN instead of a DNN.

## Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()

notebook= "M3_AST_33_CNN_Cifar_10_TFLite_C" #name of the notebook

def setup():

    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError
    else:
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError
    else:
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



### Importing required libraries

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
#import tf_keras as keras
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### Import and Inspect the Dataset

Cifar-10 repository: https://www.cs.toronto.edu/~kriz/cifar.html





In [ ]:
# From tensorflow import and download the cifar10 dataset
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
# Check for the shape of train and test images
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

- The image data shape is: `(#images, img_heigth, img_width, #channels)`, where channels are in RGB format (red, green, blue).
- The labels shape is `(#images, label)`, where label goes from 0 to 9.


In [ ]:
train_images[0]

In [ ]:
# Plotting the sample image from the dataset
plt.imshow(train_images[1]);

In [ ]:
# The CIFAR labels happen to be arrays, which is why you need the extra index
train_labels[1][0]

In [ ]:
# CIFAR10 labels/classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class_names[9] # The List's index is the label

In [ ]:
idx = train_labels[1][0]
class_names[idx]

In [ ]:
print("\t", class_names[train_labels[1][0]])
plt.imshow(train_images[1])
plt.axis('off');

### Visualize the images from trainset

In [ ]:
def plot_train_img(img, size=2):
    label = train_labels[img][0]
    plt.figure(figsize=(size,size))
    print("Label {} - {}".format(label, class_names[label]))
    plt.imshow(train_images[img])
    plt.axis('off')
    plt.show()

In [ ]:
plot_train_img(4)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

Note that images are in color, not centered and with different backgrounds

### Preprocessing the Images (Train and Test)

In [ ]:
test_images.max()

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
test_images.max()

In [ ]:
plt.hist(train_labels[:5000]);

In [ ]:
# Validation set
val_images = train_images[:5000]
val_labels = train_labels[:5000]
print(val_images.shape, val_labels.shape)

In [ ]:
train_images = train_images[5000:]
train_labels = train_labels[5000:]
print(train_images.shape, train_labels.shape)

In [ ]:
plt.hist(train_labels, alpha=0.5)
plt.hist(val_labels, alpha=0.5)
plt.hist(test_labels, alpha=0.5);

### Create CNN Model Architecture and Compile

In `tf.nn.conv2d()`: [Convolution layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

* **images** is the input mini-batch (a 4D tensor).

* **filters** is the set of filters to apply (also a 4D tensor).

* **strides** is an integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Default (1,1).

* **padding** must be either "SAME" or "VALID": Default = 'VALID'

  * If set to "SAME", the convolutional layer uses zero padding if necessary. The output size is set to the number of input neurons divided by the stride, rounded up. For example, if the input size is 13 and the stride is 5 as shown in the figure below, then the output size is 3 (i.e., 13 / 5 = 2.6, rounded up to 3). Then zeros are added as evenly as possible around the inputs, as needed.

  * If set to "VALID", the convolutional layer does not use zero padding and may
ignore some rows and columns at the bottom and right of the input image,
depending on the stride. This means that every neuron’s receptive field lies strictly within valid positions inside the input, hence the name valid.
<br><br>
<center>
<img src="https://wizardforcel.gitbooks.io/scikit-and-tensorflow-workbooks-bjpcjp/content/pics/padding-options.png" width=450px/>
</center>
$\hspace{9.5cm} \text {Different padding options}$

In [ ]:
model = Sequential()

model.add(Conv2D(
    filters=32,
    kernel_size=(3,3),
    activation='relu',
    input_shape=(32, 32, 3))
)
model.add(MaxPool2D(2, 2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
# Define the optimizer and loss
LOSS = 'sparse_categorical_crossentropy'
OPTIMIZER = 'adam'

# Compile the model
model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=['accuracy'])

### Train the model


In [ ]:
NUM_EPOCHS = 20

early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
# Fit the model
history = model.fit(train_images,
                    train_labels,
                    epochs=NUM_EPOCHS,
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stop]
)

### Visualize accuracy vs no. of epochs

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
#plt.xlim([0,NUM_EPOCHS])
plt.ylim([0.4,1.0])
plt.show()

### Evaluate the Model

In [ ]:
model.evaluate(test_images, test_labels)

**Accuracy**
- Train: 85% - 90%;
- Validation: 68%-70%
- Test: 66%-68%

In [ ]:
# Model predictions
predictions = np.argmax(model.predict(test_images), axis=-1)
predictions.shape

In [ ]:
# Print the classification report
print(classification_report(test_labels, predictions, target_names=class_names))

In [ ]:
# Print the confusion matrix
confusion_matrix(test_labels,predictions)

In [ ]:
# Plot the heatmap
plt.figure(figsize=(15,8))
sns.heatmap(confusion_matrix(test_labels,predictions), cmap='Blues', annot=True, fmt='g');

### Testing the Model (Predicting)

In [ ]:
plt.imshow(test_images[15]);

In [ ]:
test_labels[15][0]

In [ ]:
class_names[8]

In [ ]:
test_images[15].shape

The input Tensor shape should be: (num_images, width, height, color_channels)

In [ ]:
my_image = test_images[15]
my_image = my_image.reshape(1,32,32,3)
my_image.shape

In [ ]:
# Using trained CNN model to predict the image
model.predict(my_image)

In [ ]:
# Taking max probability from the predictions
img_pred = np.argmax(model.predict(my_image))
class_names[img_pred]

In [ ]:
img_pred

In [ ]:
pred_prob = model.predict(my_image)[0][img_pred]
pred_prob

### Visualizing model predictions

In [ ]:
def img_pred(img, size=4):
    label = test_labels[img][0]
    my_image = test_images[img]
    plt.figure(figsize=(size,size))
    plt.imshow(my_image)
    my_image = my_image.reshape(1,32,32,3)
    img_pred = np.argmax(model.predict(my_image))
    pred_label = class_names[img_pred]
    pred_prob = model.predict(my_image)[0][img_pred]
    print(" Label {} <=> Pred: {} with prob {:.2}".format(
        class_names[label],
        pred_label,
        pred_prob))
    plt.grid(False)
    plt.axis('off')
    plt.show()

In [ ]:
img_pred(0)

In [ ]:
for i in range (10):
  img_pred(i)

### Saving the model

In [ ]:
!pwd # Linux command, shows where we are in CoLab's folders

In [ ]:
model.save('cifar_10_model')

Use [Netron](https://netron.app) to visualize the model, hyperparameters, tensor shapes, etc. Netron is a viewer for neural network, deep learning and machine learning models (See [GitHub](https://github.com/lutzroeder/netron) for instructions about instalation in your desktop).

## Converting to TensorFlow Lite

TensorFlow Lite is TensorFlow’s lightweight solution to run TensorFlow models on mobile, embedded and IoT devices with low latency quickly. With TensorFlow Lite we can perform classification and regression task without incurring server cost.

TensorFlow Lite model supported on both Andriod and iOS via C++ and Java API. Generally, the size of the TensorFlow model is quite big that we can’t run it as it is on mobile and other embedded devices with limited compute and memory resources. So to make it possible to run TensorFlow model on low processing devices,  TensorFlow introduces a lightweight solution i.e TensorFlow Lite.

Before we start with TFLite, we need a trained model that is trained on a set of data using a high powered machine. This trained model can be converted to TensorFlow Lite format. We cannot create or train a model using TensorFlow  Lite. So, we must start with a regular TensorFlow model and then convert it to TFLite.

The next step is to convert the saved TensorFlow model to TensorFlow Lite model using TFLite converter. TensorFlow Lite model is an optimized FlatBuffer format identified by the` .tflite` file extension. Flatbuffers is a special serialization format that is optimized for performance.

### Your Task starts here

In [ ]:
# Initiate the Conversion of the saved model here
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
tflite_model = converter.convert()

In [ ]:
# Save the .tflite model
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Another way of Saving .tflite model
open("/content/cifar10.tflite","wb").write(tflite_model)

Here, the return value 673260 is the size (in bytes) of the generated TensorFlow Lite model with no quantization. We can even significantly reduce the TFLite model size by applying quantization.

### Dynamic range quantization

The simplest form of post-training quantization statically quantizes only the weights from floating point to integer, which has 8-bits of precision:

#### Convert using default quantization

Now let's enable the default `optimizations` flag to quantize all fixed parameters (such as weights):
Here, we are using saved model API. Will be useful for your custom trained model for conversion.

In [ ]:
# Initiate the Conversion of the saved model here
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set the optimization flag
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model to tflite here
tflite_quant_model = converter.convert()

At inference, weights are converted from 8-bits of precision to floating point and computed using floating-point kernels. This conversion is done once and cached to reduce latency.

To further improve latency, "dynamic-range" operators dynamically quantize activations based on their range to 8-bits and perform computations with 8-bit weights and activations. This optimization provides latencies close to fully fixed-point inference. However, the outputs are still stored using floating point so that the speedup with dynamic-range ops is less than a full fixed-point computation.

In [ ]:
# Save .tflite model
open("/content/cifar10_quant.tflite","wb").write(tflite_quant_model)

Use [Netron](https://netron.app) to visualize the quantized model. Pay attention that now the weights are int8.

### Testing TFLite Model

The converted TFLite model can be executed on mobile, embedded and IoT devices. The `TensorFlow Lite Interpreter` used to run an inference with TFLite model. `TensorFlow Lite Interpreter` is a library that takes a TFLite model file, executes the operations on input data and provide output.

Interpreter support multiple platforms and provide API to execute TFLite model from Java, C++, Python, Swift and Objective-C.

When you receive results from the model inference, you must interpret the tensors in a meaningful way that's useful in your application.

For example, a model might return only a list of probabilities. It's up to you to map the probabilities to relevant categories and present it to your end-user.

In [ ]:
# Initialize the TFLite interpreter
# Now let’s load TFLite models into Interpreter (tf.lite.Interpreter) representation, so we can run the inference process on it.
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter("/content/cifar10_quant.tflite")

tflite_model can be saved to a file and loaded later, or directly into the Interpreter. Since TensorFlow Lite pre-plans tensor allocations to optimize inference, the user needs to call allocate_tensors() before any inference.

In [ ]:
# Load the TFLite model and allocate tensors
interpreter.allocate_tensors() # Needed before execution!

# Get input and output tensors.
input_details = interpreter.get_input_details()   # Model has single input
output_details = interpreter.get_output_details() # Model has single output

The `input_details` represents the pieces of information related to input data formats such as the shape of input data, the data type of input and other various information related to quantization. And output_details contains information related to the model output format.

In [ ]:
input_details

Gets model input tensor details.

Returns:

A list in which each item is a dictionary with details about an input tensor. Each dictionary contains the following fields that describe the tensor:
* name: The tensor name.

* index: The tensor index in the interpreter.

* shape: The shape of the tensor.

* shape_signature: Same as shape for models with known/fixed shapes. If any dimension sizes are unkown, they are indicated with -1.

* dtype: The numpy data type (such as np.int32 or np.uint8).

* quantization: Deprecated, use quantization_parameters. This field only works for per-tensor quantization, whereas quantization_parameters works in all cases.

* quantization_parameters: A dictionary of parameters used to quantize the tensor: ~ scales: List of scales (one if per-tensor quantization). ~ zero_points: List of zero_points (one if per-tensor quantization). ~ quantized_dimension: Specifies the dimension of per-axis quantization, in the case of multiple scales/zero_points.

* sparsity_parameters: A dictionary of parameters used to encode a sparse tensor. This is empty if the tensor is dense.

In [ ]:
output_details

Gets model output tensor details.

Returns: A list in which each item is a dictionary with details about an output tensor.

In [ ]:
def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

In [ ]:
image = test_images[0]
plt.imshow(image);

In [ ]:
set_input_tensor(interpreter, image)

# Run inference.
interpreter.invoke()
output_details = interpreter.get_output_details()[0]

In [ ]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
interpreter.get_tensor(output_details['index'])

get_tensor(
    tensor_index
) : Gets the value of the output tensor (get a copy).

If you wish to avoid the copy, use tensor(). This function cannot be used to read intermediate results.

Args:

tensor_index: Tensor index of tensor to get. This value can be gotten from the 'index' field in get_output_details.

Returns
a numpy array.

In [ ]:
np.squeeze(interpreter.get_tensor(output_details['index']))

In [ ]:
output = np.squeeze(interpreter.get_tensor(output_details['index']))
output

In [ ]:
output.shape

In [ ]:
img_pred = np.argmax(output)
class_names[img_pred]

In [ ]:
img_pred

In [ ]:
output[img_pred]

In [ ]:
def classify_image(image):
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    img_pred = np.argmax(output)
    pred_label = class_names[img_pred]
    pred_prob = output[img_pred]
    plt.imshow(image)
    print(" Pred: {} with prob {:.2}".format(
        pred_label,
        pred_prob))
    plt.grid(False)
    plt.axis('off')
    plt.show()

In [ ]:
classify_image(test_images[0])

In [ ]:
for i in range (10):
  classify_image(test_images[i])

## Initiate TFlite optimization

**Note:** Refer the following [link ](https://www.tensorflow.org/model_optimization/)

In [ ]:
!pip install -q tensorflow-model-optimization
!pip install -q tensorflow
, quantized_layer_name_prefix='quant_
#pip install -q tensorflow-model-optimization

In [ ]:
import numpy as np
import tempfile
import zipfile
import os

 Evaluate the baseline model and save it for later usage

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

### Prune and fine-tune the model to 50% sparsity


Apply the prune_low_magnitude() API to prune the whole pre-trained model to achieve the model that is to be clustered in the next step. For how best to use the API to achieve the best compression rate while maintaining your target accuracy, refer to the [pruning comprehensive guide](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide).

#### Define the model and apply the sparsity API


To make the whole model train with pruning, apply `tfmot.sparsity.keras.prune_low_magnitude` to the model.

In [ ]:
import tensorflow_model_optimization as tfmot

# define a list containing the names or types of the
# layers that shall be pruned, and iterate the layer-wise pruning over all layers in this list
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude # Provide the type of sparsity API

# Pruning schedule with constant sparsity(%) throughout training
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=10) # Provide the sparsity value
  }

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep() # Keras callback which updates pruning wrappers with the optimizer step and is required during training.
]

pruned_model = prune_low_magnitude(model, **pruning_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

# Compile the pruned model
pruned_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

In [ ]:
# Print the pruned model
pruned_model.summary()

### Fine-tune the model, check sparsity, and evaluate the accuracy against baseline

Fine-tune the model with pruning for 3 epochs.

In [ ]:
# Fine-tune model
pruned_model.fit(
  train_images,
  train_labels,
  epochs=3,
  validation_split=0.1,
  callbacks=callbacks)

### Define helper functions to calculate and print the sparsity and clusters of the model.

In [ ]:
def print_model_weights_sparsity(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

# Define helper functions to calculate and print the number of clustering in each kernel of the model.
def print_model_weight_clusters(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            # ignore auxiliary quantization weights
            if "quantize_layer" in weight.name:
                continue
            if "kernel" in weight.name:
                unique_count = len(np.unique(weight))
                print(
                    f"{layer.name}/{weight.name}: {unique_count} clusters "
                )

Let's strip the pruning wrapper first, then check that the model kernels were correctly pruned.

 `tfmot.sparsity.keras.strip_pruning` Strip pruning wrappers from the model and is necessary to see the compression
benefits of pruning.

*   `strip_pruning:` Once a model has been pruned to required sparsity, this method can be used to restore the original model with the sparse weights.


First, create a compressible model for TensorFlow.

In [ ]:
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model) # Pass the pruned model

print_model_weights_sparsity(stripped_pruned_model)

### Apply sparsity preserving clustering and check its effect on model sparsity in both cases

Next, we apply both clustering and sparsity preserving clustering on the pruned model and observe the number of clusters and check that the sparsity is preserved on the pruned model.  Generate a TFLite model and check that the accuracy has been preserved in the pruned clustered model.

Clustering, or weight sharing, reduces the number of unique weight values in a model, leading to benefits for deployment. It first groups the weights of each layer into N clusters, then shares the cluster's centroid value for all the weights belonging to the cluster. This technique brings improvements via model compression.


Note that we stripped pruning wrappers from the pruned model with tfmot.sparsity.keras.strip_pruning before applying the clustering API.

### Fine-tune the pre-trained model with clustering

Apply the cluster_weights() API to a whole pre-trained model to demonstrate its effectiveness in reducing the model size after applying zip while keeping decent accuracy.

#### Define the model and apply the clustering API

Before you pass the model to the clustering API, make sure it is trained and shows some acceptable accuracy.

In [ ]:
import tensorflow_model_optimization as tfmot

# Sparsity preserving clustering
from tensorflow_model_optimization.python.core.clustering.keras.experimental import (
    cluster,
)

# Clustering
cluster_weights = tfmot.clustering.keras.cluster_weights # Provide the type of sparsity API
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

cluster_weights = cluster.cluster_weights

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS,
  'preserve_sparsity': True
}

# Cluster a whole model
sparsity_clustered_model = cluster_weights(stripped_pruned_model, **clustering_params)

# Compile the model
sparsity_clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the sparsity preserving clustering model
print('Train sparsity preserving clustering model:')
sparsity_clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)

Strip the clustering wrapper first, then check that the model is correctly pruned and clustered.

In [ ]:
stripped_clustered_model = tfmot.clustering.keras.strip_clustering(sparsity_clustered_model)

print("Model sparsity:\n")
print_model_weights_sparsity(stripped_clustered_model)

print("\nModel clusters:\n")
print_model_weight_clusters(stripped_clustered_model)

### Apply QAT (Quantization Aware Training) and PCQAT (Preserving Quantization Aware Training) and check effect on model clusters and sparsity


Next, apply both QAT and PCQAT on the sparse clustered model and observe that PCQAT preserves weight sparsity and clusters in your model. Note that the stripped model is passed to the QAT and PCQAT API.

Since this model is going to be deployed on a microcontroller, we want it to be as tiny as possible! One technique for reducing the size of a model is called quantization. It reduces the precision of the model's weights, and possibly the activations (output of each layer) as well, which saves memory, often without much impact on accuracy. Quantized models also run faster, since the calculations required are simpler.

You will apply quantization aware training to the whole model and see this in the model summary. All layers are now prefixed by "quant".

Note that the resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8).


In [ ]:
# QAT
qat_model = tfmot.quantization.keras.quantize_annotate_model(stripped_clustered_model) # Apply Quantization on stripped clustered model
#qat_model = tfmot.quantization.keras.quantize_apply(stripped_clustered_model1) # Apply Quantization on stripped clustered model

# Compile the QAT model
qat_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the QAT model
print('Train qat model:')
qat_model.fit(train_images, train_labels, batch_size=128, epochs=3, validation_split=0.1)

# PCQAT
# Annotate a model (stripped_clustered_model) to be quantized
# This function does not actually quantize the model. It merely specifies that the
# model needs to be quantized. 'quantize_apply' can then be used to quantize the model.
pcqat_model = tfmot.quantization.keras.quantize_annotate_model(
              stripped_clustered_model)
#pcqat_model = tfmot.quantization.keras.quantize_model(
#              quant_aware_annotate_model,
#              # Default 8 bit Cluster Preserve Quantization Scheme
#              # preserve_sparsity - the flag to enable prune-cluster-preserving QAT
#              tfmot.experimental.combine.Default8BitClusterPreserveQuantizeScheme(preserve_sparsity=True))

pcqat_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print('Train pcqat model:')
pcqat_model.fit(train_images, train_labels, batch_size=128, epochs=3, validation_split=0.1)

In [ ]:
print("QAT Model clusters:")
print_model_weight_clusters(qat_model)
print("\nQAT Model sparsity:")
print_model_weights_sparsity(qat_model)
print("\nPCQAT Model clusters:")
print_model_weight_clusters(pcqat_model)
print("\nPCQAT Model sparsity:")
print_model_weights_sparsity(pcqat_model)

### See compression benefits of PCQAT model


Define a helper function to actually compress the models via gzip and measure the zipped size.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in kilobytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)/1000

Observe that applying sparsity, clustering and PCQAT to a model yields significant compression benefits.

In [ ]:
# QAT model
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model) # Initiate the Conversion of the QAT model here
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Provide the default optimization option
qat_tflite_model = converter.convert() # Conversion to tflite
qat_model_file = 'qat_model.tflite'

# Save the model
# Provide the save model filename and code
with open(qat_model_file, 'wb') as f:
    f.write(qat_tflite_model)

# PCQAT model
converter = tf.lite.TFLiteConverter.from_keras_model(pcqat_model) # Initiate the Conversion of the PCQAT model here
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Provide the default optimization option
pcqat_tflite_model = converter.convert() # Conversion to tflite
pcqat_model_file = 'pcqat_model.tflite'

# Save the model
# Provide the save model filename and code
with open(pcqat_model_file, 'wb') as f:
    f.write(pcqat_tflite_model)

print("QAT model size: ", get_gzipped_model_size(qat_model_file), ' KB')
print("PCQAT model size: ", get_gzipped_model_size(pcqat_model_file), ' KB')

### See the persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TFLite model on the test dataset.


In [ ]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print(f"Evaluated on {i} results so far.")
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Evaluate the model, which has been pruned, clustered and quantized, and then see that the accuracy from TensorFlow persists in the TFLite backend.

In [ ]:
interpreter = tf.lite.Interpreter(pcqat_model_file)
interpreter.allocate_tensors()

pcqat_test_accuracy = eval_model(interpreter)

print('Pruned, clustered and quantized TFLite test_accuracy:', pcqat_test_accuracy)
print('Baseline TF test accuracy:', baseline_model_accuracy)

# TensorFlow Lite Micro

### Generate a TensorFlow Lite for Microcontrollers Model
To convert the TensorFlow Lite quantized model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers on MCUs, we simply need to use the ```xxd``` tool to convert the ```.tflite``` file into a ```.cc``` file.

**Convert to a C array**

First install xxd

In [ ]:
!apt-get update && apt-get -qq install xxd

Now, convert and save the .cc converted model

In [ ]:
MODEL_TFLITE = 'cifar10_quant.tflite'
MODEL_TFLITE_MICRO = 'cifar10_quant_model.cc'
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

If you'd like to download your model for safekeeping:
1. On the left of the UI click on the folder icon
2. Click on the three dots to the right of the ```cifar10_quant_model.cc``` file and select download

In [ ]:
!cat {MODEL_TFLITE_MICRO}

### Please answer the questions below to complete the experiment:




#### Consider the statements given below and answer Q1.

A. Quantization works by reducing the precision of the numbers used to represent a model's parameters, which by default are 32-bit floating point numbers. This results in a smaller model size and faster computation.

B. Pruning works by removing parameters within a model that have only a minor impact on its predictions.

C. Clustering works by grouping the weights of each layer in a model into a predefined number of clusters, then sharing the centroid values for the weights belonging to each individual cluster. This reduces the number of unique weight values in a model, thus reducing its complexity.



In [ ]:
#@title Q.1. Which of the above statements is/are True about TensorflowLite Model Optimization?
Answer1 = "" #@param ["","Only A", "Only B", "Only C", "Only A and B","Only B and C", "A, B, C"]


In [ ]:
#@title Q.2. Magnitude-based weight pruning gradually zeroes out model weights during the training process to achieve model sparsity. Sparse models are easier to compress, and we can skip the zeroes during inference for latency improvements.
Answer2 = "" #@param ["","TRUE","FALSE"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")